# 전처리

In [ ]:
## 전처리 프로세스

1. 제품 결측값 -> 0 대체 (보유하지 않음으로 가정)
2. 카테고리 변수 -> label encoding


In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2023)

In [6]:
train = pd.read_csv('../../data/train_ver2.csv')


/var/folders/0x/y5ht6wyj4ql4by49_d742qs00000gn/T/ipykernel_41079/3262364342.py:1: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../../data/train_ver2.csv')


In [8]:
test = pd.read_csv('../../data/test_ver2.csv')
test

/var/folders/0x/y5ht6wyj4ql4by49_d742qs00000gn/T/ipykernel_41079/2557336278.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('../../data/test_ver2.csv')


,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,...,N,N,KAT,N,1,28.0,MADRID,1,326124.90,01 - TOP
1,2016-06-28,1170544,N,ES,H,36,2013-08-28,0,34,1,...,N,NaN,KAT,N,1,3.0,ALICANTE,0,NA,02 - PARTICULARES
2,2016-06-28,1170545,N,ES,V,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,15.0,"CORUÑA, A",1,NA,03 - UNIVERSITARIO
3,2016-06-28,1170547,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,8.0,BARCELONA,0,148402.98,03 - UNIVERSITARIO
4,2016-06-28,1170548,N,ES,H,22,2013-08-28,0,34,1,...,N,NaN,KHE,N,1,7.0,"BALEARS, ILLES",0,106885.80,03 - UNIVERSITARIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929610,2016-06-28,660237,N,ES,V,55,1999-04-21,0,206,1,...,N,NaN,KAT,N,1,28.0,MADRID,1,128643.57,01 - TOP
929611,2016-06-28,660238,N,ES,V,30,2006-11-29,0,115,1,...,N,NaN,KFC,N,1,26.0,"RIOJA, LA",0,NA,02 - PARTICULARES
929612,2016-06-28,660240,N,ES,V,52,2006-11-29,0,115,1,...,N,NaN,KBZ,N,1,33.0,ASTURIAS,1,72765.27,02 - PARTICULARES
929613,2016-06-28,660243,N,ES,V,32,2006-11-29,0,115,1,...,N,NaN,KFC,N,1,33.0,ASTURIAS,0,147488.88,02 - PARTICULARES


In [9]:
prods = [col for col in train.columns if 'ult1' in col]

In [10]:
## 결측치 처리
train[prods] = train[prods].fillna(0.0).astype(np.int8)



In [11]:
## 모든 제품을 보유하지 않은 고객은 제거
no_product_df = train[prods].sum(axis=1) == 0
train = train[~no_product_df]
train.shape

(11091070, 48)

In [12]:
## 학습 + 테스트 데이터 통합
for col in prods:
    test[col] = 0

test

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2016-06-28,15889,F,ES,V,56,1995-01-16,0,256,1,...,0,0,0,0,0,0,0,0,0,0
1,2016-06-28,1170544,N,ES,H,36,2013-08-28,0,34,1,...,0,0,0,0,0,0,0,0,0,0
2,2016-06-28,1170545,N,ES,V,22,2013-08-28,0,34,1,...,0,0,0,0,0,0,0,0,0,0
3,2016-06-28,1170547,N,ES,H,22,2013-08-28,0,34,1,...,0,0,0,0,0,0,0,0,0,0
4,2016-06-28,1170548,N,ES,H,22,2013-08-28,0,34,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929610,2016-06-28,660237,N,ES,V,55,1999-04-21,0,206,1,...,0,0,0,0,0,0,0,0,0,0
929611,2016-06-28,660238,N,ES,V,30,2006-11-29,0,115,1,...,0,0,0,0,0,0,0,0,0,0
929612,2016-06-28,660240,N,ES,V,52,2006-11-29,0,115,1,...,0,0,0,0,0,0,0,0,0,0
929613,2016-06-28,660243,N,ES,V,32,2006-11-29,0,115,1,...,0,0,0,0,0,0,0,0,0,0


In [13]:
total_df = pd.concat([train, test], axis=0)

In [14]:
total_df.reset_index(drop=True, inplace=True)

In [15]:
total_df

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12020680,2016-06-28,660237,N,ES,V,55,1999-04-21,0.0,206,1.0,...,0,0,0,0,0,0,0,0,0,0
12020681,2016-06-28,660238,N,ES,V,30,2006-11-29,0.0,115,1.0,...,0,0,0,0,0,0,0,0,0,0
12020682,2016-06-28,660240,N,ES,V,52,2006-11-29,0.0,115,1.0,...,0,0,0,0,0,0,0,0,0,0
12020683,2016-06-28,660243,N,ES,V,32,2006-11-29,0.0,115,1.0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
## 학습 사용 변수
features = []

In [17]:
## label encoding
cat_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp',
           'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in cat_cols:
    print(col)
    total_df[col], _ = total_df[col].factorize(na_sentinel=-99)
    

ind_empleado
pais_residencia
sexo
tiprel_1mes
indresi
indext
conyuemp
canal_entrada
indfall
tipodom
nomprov
segmento


In [18]:
features += cat_cols

In [19]:

total_df['age'].replace(' NA', -99, inplace=True)
total_df['age'] = total_df['age'].astype(np.int8)


In [20]:
total_df['antiguedad'] = total_df['antiguedad'].replace('     NA', -99).astype(np.int8)

In [21]:
total_df['renta'].fillna(-99, inplace=True)
total_df['renta'].replace('         NA', -99, inplace=True)

In [22]:
total_df['indrel_1mes'].replace('P', 5, inplace=True)
total_df['indrel_1mes'].fillna(-99, inplace=True)
total_df['indrel_1mes'] = total_df['indrel_1mes'].astype(float).astype(np.int8)


In [23]:
features += ['age', 'antiguedad', 'renta', 'ind_nuevo', 'indrel', 'indrel_1mes',
            'ind_actividad_cliente']

# 피쳐 엔지니어링

In [24]:
## date변수 -> 연도, 월 추출

# fecha_alta : 첫 가입일
total_df['apply_month'] = total_df['fecha_alta'].map(lambda x: 0.0 if type(x)==float else x.split('-')[1]).astype(np.int8)
total_df['apply_year'] = total_df['fecha_alta'].map(lambda x: 0.0 if type(x)==float else x.split('-')[0]).astype(np.int16)
features += ['apply_month', 'apply_year']


In [25]:
#ult_fec_cli_1t : 마지막 1등급 날짜
total_df['last_1_month'] = total_df['ult_fec_cli_1t'].map(lambda x: 0.0 if type(x)==float else x.split('-')[1]).astype(np.int8)
total_df['last_1_year'] = total_df['ult_fec_cli_1t'].map(lambda x: 0.0 if type(x)==float else x.split('-')[0]).astype(np.int16)
features += ['last_1_month', 'last_1_year']



In [26]:
total_df.fillna(-99, inplace=True)

In [27]:
## lag 데이터 생성

# 날짜 값을 정수로 변환
def date_to_int(str_date):
    y, m, d= [int(x) for x in str_date.strip().split('-')]
    int_date = (y - 2015) *12 + m
    return int_date

total_df['int_date'] = total_df['fecha_dato'].map(date_to_int).astype(np.int8)


In [28]:

# 한달전 값을 변수로 가지도록 

df_lag = total_df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in total_df.columns]
df_lag['int_date'] += 1

df = total_df.merge(df_lag, on=['ncodpers', 'int_date'], how='left')


In [29]:
del df_lag, total_df

In [30]:
# 이전달 제품 정보가 존재하지 않을 경우 0 (가입하지 않은 상태로 가정)
for prod in prods:
    prev = prod + '_prev'
    df[prev].fillna(0, inplace=True)

df.fillna(-99, inplace=True)

In [31]:
# lag 변수 추가
features += [f + '_prev' for f in features]
features += [prod + '_prev' for prod in prods]

# 데이터 저장

In [32]:
import pickle

with open('../../data/baseline_feats.pkl', 'wb') as f:
    pickle.dump(features, f)

In [34]:
df.to_csv('../../data/baseline_df.csv', index=False)